# Bidding

This is a tutorial for how to use the bidding engine

In [1]:
import os
os.chdir('..')

from nn.models import Models
from bots import BotBid
from bidding import bidding
from util import hand_to_str
from deck52 import random_deal
from sample import Sample
import conf
import numpy as np

np.set_printoptions(precision=2, suppress=True, linewidth=200)


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
models = Models.from_conf(conf.load('./config/default.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('./config/default.conf'))  # Load sampling strategies

INFO:tensorflow:Restoring parameters from ..\UCBC 2024/Models/NS1EW99-bidding_same-278000
INFO:tensorflow:Restoring parameters from ..\UCBC 2024/Models/NS1EW99-binfo_same-278000
INFO:tensorflow:Restoring parameters from ..\UCBC 2024/Models/lead_suit-154000
INFO:tensorflow:Restoring parameters from ..\UCBC 2024/Models/lead_nt-59000
INFO:tensorflow:Restoring parameters from ..\UCBC 2024/Models/lr3-1000000
INFO:tensorflow:Restoring parameters from ..\UCBC 2024/Models/single_dummy-32768000
INFO:tensorflow:Restoring parameters from ..\UCBC 2024/Models/lefty-1000000
INFO:tensorflow:Restoring parameters from ..\UCBC 2024/Models/dummy-920000
INFO:tensorflow:Restoring parameters from ..\UCBC 2024/Models/righty-1000000
INFO:tensorflow:Restoring parameters from ..\UCBC 2024/Models/decl-1000000


### Running through an example

In [3]:

vuln_ns, vuln_ew = False, False

# you sit West and hold:
hand = '6.AKJT82.762.K63'

# the auction goes:
# (a few words about 'PAD_START':
# the auction is padded to dealer North
# if North is not dealer, than we have to put in a 'PAD_START' for every seat that was skipped
# if East deals we have one pad (because North is skipped)
# if South deals we have two pads (because North and East are skipped)
# etc.)
auction = ['1D', '3S']
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, sampler, 2, 0, False)

bid = bot_bid.bid(auction)
bid.to_dict()['candidates']

# what's your bid?

[{'call': '4H', 'insta_score': 0.188, 'expected_score': 186, 'adjustment': 9},
 {'call': 'X', 'insta_score': 0.433, 'expected_score': 131, 'adjustment': 22},
 {'call': 'PASS',
  'insta_score': 0.354,
  'expected_score': 122,
  'adjustment': 18}]

In [4]:
auction = ['1D', '3S', '4H', '4S', '5H', 'PASS']
bid = bot_bid.bid(auction)
bid.to_dict()['candidates']

[{'call': 'PASS', 'insta_score': 0.961}]

In [5]:
# you double, North bids 2S, your partner 3H, PASS, back to you ...

bid = bot_bid.bid(["PAD_START", "PASS", "1H", "X", "4H", "PASS", "PASS", "X"])
bid.bid

['PAD_START', 'PASS', '1H', 'X', '4H', 'PASS', 'PASS', 'X']
Bid not valid: 4H insta_score: 0.218 0.1


'PASS'

In [6]:
# what options were considered?

bid.to_dict()['candidates']

[{'call': 'PASS', 'insta_score': 0.219}]

in the output above:
- `insta_score` reflects what the neural network prefers to bid
- `expected_score` is based on what you expect to score on difference samples (for example, 70% of the time you are making and 30% of the time you are going one off, then the expected score is 0.7 * 620 - 0.3 * 100)

### Samples consistent with the auction so far

Above we said that we are computig expected scores on samples consistent with the auction so far.

How do we get these samples?

We get some samples in the response we get from the bot (i.e `bid.samples`)

In [7]:
bid.samples

[]

each row is one sample. the hands are in the order: N E S W

(West always has the same cards because it's your hand and it's known to you)

In [8]:
# we got some samples above. if we want more, we can always get more

sample_hands, sorted_score, p_hcp, p_shp, good_quality = bot_bid.sample_hands_for_auction(['PAD_START', 'PAD_START', 'PAD_START','2N','PASS','3C','PASS','3D'], 0)
print("Good quality:", good_quality)
for i in range(sample_hands.shape[0]):
    print(f'{hand_to_str(sample_hands[i, 0, :])} {hand_to_str(sample_hands[i, 1, :])} {hand_to_str(sample_hands[i, 2, :])} {hand_to_str(sample_hands[i, 3, :])}')

Good quality: True
x.AKJT8x.xxx.Kxx Q9xx.9xxx.KJ.Txx JT8xx.x.8xxx.Q8x AKx.Qx.AQT9.AJ9x
x.AKJT8x.xxx.Kxx Qxxx.9x.J9x.Qxxx JT98x.xxx.Kx.98x AKx.Qx.AQT8x.AJT
x.AKJT8x.xxx.Kxx AT9x.Q9xx.8xx.Q8 J8xxx.x.T9x.T9xx KQx.xx.AKQJ.AJxx
x.AKJT8x.xxx.Kxx K9xx.xx.Tx.Q8xxx Q8xxx.9x.J8x.T9x AJT.Qxx.AKQ9x.AJ
x.AKJT8x.xxx.Kxx QJ9x.xx.AJ9.Jxxx T8xxx.9x.xxx.T98 AKx.Qxx.KQT8.AQx
x.AKJT8x.xxx.Kxx Q9xx.Qx.Qxx.Jxxx JTxxx.xx.JT9.T8x AK8.9xx.AK8x.AQ9
x.AKJT8x.xxx.Kxx Qxxx.9xxx.98.AJx JT98x.x.JTxx.Txx AKx.Qx.AKQx.Q98x
x.AKJT8x.xxx.Kxx K9x.Qxxx.8xx.8xx QJT8xx.x.T9x.T9x Axx.9x.AKQJ.AQJx
x.AKJT8x.xxx.Kxx KTxx.xx.QTxx.Q9x Q98xx.xxx.9x.8xx AJx.Q9.AKJ8.AJTx
x.AKJT8x.xxx.Kxx KQ8x.xxx.JTx.Qxx T9xxx.9.98x.J8xx AJx.Qxx.AKQx.AT9
x.AKJT8x.xxx.Kxx A9xx.9xx.xx.QJ8x JT8xx.xx.JT98x.x KQx.Qx.AKQ.AT9xx
x.AKJT8x.xxx.Kxx KT8x.xx.QJTx.T9x Q9xxx.9x.98x.Jxx AJx.Qxx.AKx.AQ8x
x.AKJT8x.xxx.Kxx QT8xx.Qxxx.Kxx.x J9xx.x.T98x.98xx AKx.9x.AQJ.AQJTx
x.AKJT8x.xxx.Kxx T9xx.xxxx.Q8xx.Q QJ8xx.9.J9x.J8xx AKx.Qx.AKT.AT9xx
x.AKJT8x.xxx.Kxx AT8x.xxx.9.T

How do we know on which sample 4H is making and on which sample it is not? It's the topic for another tutorial, but basically we could do either (1) solve the sample double dummpy or (2) estimate the outcome using a neural network

### Bidding through an entire auction for all four hands

Now we'll take a random deal and let the bot bid for each hand. We assume North deals and nobody is vulnerable.

In [9]:
hands = random_deal().split()
hands # N, E, S, W

['QT942..KJT6.T963',
 'AK8753.QT5.43.J5',
 'J.764.AQ9875.A84',
 '6.AKJ9832.2.KQ72']

In [10]:
bidder_bots = [BotBid([False, False], hand, models, sampler,i, 0,False) for i, hand in enumerate(hands)]

In [11]:
auction = []  # since North deals, we don't need any 'PAD_START'

turn_i = 0  # whose turn is it to bid

while not bidding.auction_over(auction):
    auction.append(bidder_bots[turn_i].bid(auction).bid)
    turn_i = (turn_i + 1) % 4  # next player's turn
    
auction

['PASS', '2S', 'PASS', '3H', 'PASS', '4H', 'PASS', 'PASS', 'PASS']